# UFoE Phase 5b-2: ColabFold Multimer 최종 검증
## Claude + GenSpark 통합본

### 확인 사항:
1. **Inter-chain salt bridge** (e↔g' 정규/비정규 분리)
2. **a,d core packing** (Cβ-Cβ distance, d↔d'/a↔a'/a↔d' 3축)
3. **True crossing angle** (DSSP helix-only PCA)
4. **Oligomeric state** (C 그룹 dimer 유지 vs collapse)
5. **SASA** (곤 매몰 최종 확인)

### 실험 그룹:
| Group | 서열 | 설계 | 예상 |
|-------|------|------|------|
| A (WT) | GCN4 원본 33aa | 자연 균형 | crossing 18-25° |
| B (곤↑) | a,d→L/I 100% | 코어 강화 | d↔d' 최밀 |
| C (건↑) | e→E, g→K | 표면 과다 | 코어 붕괴 or 약화 |

---

In [ ]:
# Step 1: 설치
!pip install -q colabfold[alphafold]@git+https://github.com/sokrypton/ColabFold
!pip install -q biopython numpy scipy matplotlib
!pip install -q py3Dmol

In [ ]:
# Step 2: 확정 서열 + 곤감리건 검증
import os, json, glob
import numpy as np
from pathlib import Path

SEQUENCES = {
    'A_WT':   'RMKQLEDKVEELLSKNYHLENEVARLKKLVGER',
    'B_GON':  'LMKQLEDIVEELLSLNYHLEINEVALKKLLGEL',
    'C_GEON': 'REKSLEDKVEELLSKEYKLENEVERKKLEGEK'
}

GGRL_MAP = {
    'A':'gon','L':'gon','V':'gon','I':'gon','F':'gon','M':'gon','W':'gon',
    'G':'gam','C':'gam','P':'gam',
    'S':'ri','T':'ri','N':'ri','Q':'ri','H':'ri','Y':'ri',
    'E':'geon','D':'geon','K':'geon','R':'geon'
}

def get_heptad_registry(seq_length, start_position='d'):
    heptad = ['d','e','f','g','a','b','c']
    off = heptad.index(start_position)
    return {i: heptad[(i + off) % 7] for i in range(seq_length)}

def get_positions_by_role(registry):
    roles = {'a':[],'d':[],'e':[],'g':[],'bcf':[]}
    for idx, letter in registry.items():
        if letter in ('a','d','e','g'):
            roles[letter].append(idx)
        else:
            roles['bcf'].append(idx)
    return roles

# 서열 검증
for name, seq in SEQUENCES.items():
    reg = get_heptad_registry(len(seq))
    pos = get_positions_by_role(reg)
    ggrl = [GGRL_MAP[aa] for aa in seq]
    n = len(ggrl)
    ratio = {t: round(ggrl.count(t)/n*100,1) for t in ['gon','gam','ri','geon']}
    print(f'{name} ({len(seq)}aa): gon={ratio["gon"]}% gam={ratio["gam"]}% ri={ratio["ri"]}% geon={ratio["geon"]}%')
    print(f'  a: {[seq[i] for i in pos["a"]]}  d: {[seq[i] for i in pos["d"]]}')
    print(f'  e: {[seq[i] for i in pos["e"]]}  g: {[seq[i] for i in pos["g"]]}')
    print()

In [ ]:
# Step 3: FASTA 생성 + ColabFold 실행
os.makedirs('gcn4_colabfold', exist_ok=True)

for name, seq in SEQUENCES.items():
    fasta = f'gcn4_colabfold/{name}.fasta'
    with open(fasta, 'w') as f:
        f.write(f'>{name}\n{seq}:{seq}\n')
    print(f'{name}: {fasta}')

print('\n--- ColabFold 실행 ---')
for name in SEQUENCES:
    fasta = f'gcn4_colabfold/{name}.fasta'
    outdir = f'gcn4_colabfold/output_{name}'
    os.makedirs(outdir, exist_ok=True)
    print(f'\n=== {name} ===')
    !colabfold_batch {fasta} {outdir} --num-models 5 --num-recycles 3 --model-type alphafold2_multimer_v3 --amber
    print(f'Done: {name}')

In [ ]:
# Step 4: PDB 로드 (5개 모델 전부)
from Bio.PDB import PDBParser

def find_all_pdbs(output_dir, max_models=5):
    """rank 순서로 PDB 파일 최대 5개 반환"""
    pdbs = sorted(glob.glob(os.path.join(output_dir, '*rank_*.pdb')))
    if not pdbs:
        pdbs = sorted(glob.glob(os.path.join(output_dir, '*.pdb')))
    return pdbs[:max_models]

pdb_files = {}
for name in SEQUENCES:
    outdir = f'gcn4_colabfold/output_{name}'
    pdbs = find_all_pdbs(outdir)
    pdb_files[name] = pdbs
    print(f'{name}: {len(pdbs)} models found')
    
    # 체인 수 확인 (첫 번째 모델)
    if pdbs:
        parser = PDBParser(QUIET=True)
        s = parser.get_structure(name, pdbs[0])
        chains = list(s[0].get_chains())
        print(f'  Chains: {len(chains)} ({[c.id for c in chains]})')

---
## 분석 함수 정의

In [ ]:
# === 분석 1: Salt Bridge (e↔g' registry 필터) ===

def detect_salt_bridges(pdb_file, sequence, cutoff=4.0):
    parser = PDBParser(QUIET=True)
    structure = parser.get_structure('d', pdb_file)
    chains = list(structure[0].get_chains())
    if len(chains) < 2:
        return None
    
    registry = get_heptad_registry(len(sequence))
    positions = get_positions_by_role(registry)
    e_set, g_set = set(positions['e']), set(positions['g'])
    
    neg_atoms = {'GLU':['OE1','OE2'], 'ASP':['OD1','OD2']}
    pos_atoms = {'LYS':['NZ'], 'ARG':['NH1','NH2']}
    
    regular, irregular = [], []
    res_a = list(chains[0].get_residues())
    res_b = list(chains[1].get_residues())
    
    # 양방향: A(neg)↔B(pos) + A(pos)↔B(neg)
    pairs = []
    for i, ra in enumerate(res_a):
        rn_a = ra.get_resname()
        for j, rb in enumerate(res_b):
            rn_b = rb.get_resname()
            # neg↔pos
            if rn_a in neg_atoms and rn_b in pos_atoms:
                pairs.append((i, ra, rn_a, neg_atoms[rn_a], j, rb, rn_b, pos_atoms[rn_b]))
            # pos↔neg
            if rn_a in pos_atoms and rn_b in neg_atoms:
                pairs.append((i, ra, rn_a, pos_atoms[rn_a], j, rb, rn_b, neg_atoms[rn_b]))
    
    for i, ra, rn_a, a_names, j, rb, rn_b, b_names in pairs:
        min_d = 999
        for an in a_names:
            for bn in b_names:
                if an in ra and bn in rb:
                    d = ra[an] - rb[bn]
                    if d < min_d: min_d = d
        
        if min_d < cutoff:
            info = {
                'a_res': f'{rn_a}{i+1}', 'b_res': f'{rn_b}{j+1}',
                'dist': round(min_d, 2),
                'heptad_a': registry.get(i,'?'), 'heptad_b': registry.get(j,'?')
            }
            is_eg = (i in e_set and j in g_set) or (i in g_set and j in e_set)
            if is_eg:
                info['type'] = 'REGULAR'
                regular.append(info)
            else:
                info['type'] = 'irregular'
                irregular.append(info)
    
    return {'regular': regular, 'irregular': irregular,
            'regular_count': len(regular), 'irregular_count': len(irregular),
            'total': len(regular) + len(irregular)}

print('Salt bridge function loaded.')

In [ ]:
# === 분석 2: Core Packing (d↔d', a↔a', a↔d' 3축) ===

def analyze_core_packing(pdb_file, sequence):
    parser = PDBParser(QUIET=True)
    structure = parser.get_structure('d', pdb_file)
    chains = list(structure[0].get_chains())
    if len(chains) < 2:
        return None
    
    registry = get_heptad_registry(len(sequence))
    positions = get_positions_by_role(registry)
    
    def get_cb(res):
        if 'CB' in res: return res['CB'].get_vector().get_array()
        elif 'CA' in res: return res['CA'].get_vector().get_array()
        return None
    
    ra = list(chains[0].get_residues())
    rb = list(chains[1].get_residues())
    
    dists = {'d_d':[], 'a_a':[], 'a_d':[]}
    details = {'d_d':[], 'a_a':[]}
    
    for pos in positions['d']:
        if pos < len(ra) and pos < len(rb):
            ca, cb = get_cb(ra[pos]), get_cb(rb[pos])
            if ca is not None and cb is not None:
                d = round(np.linalg.norm(ca - cb), 2)
                dists['d_d'].append(d)
                details['d_d'].append({'pos':pos+1, 'aa':sequence[pos], 'dist':d})
    
    for pos in positions['a']:
        if pos < len(ra) and pos < len(rb):
            ca, cb = get_cb(ra[pos]), get_cb(rb[pos])
            if ca is not None and cb is not None:
                d = round(np.linalg.norm(ca - cb), 2)
                dists['a_a'].append(d)
                details['a_a'].append({'pos':pos+1, 'aa':sequence[pos], 'dist':d})
    
    for pa in positions['a']:
        for pd in positions['d']:
            if pa < len(ra) and pd < len(rb):
                ca, cb = get_cb(ra[pa]), get_cb(rb[pd])
                if ca is not None and cb is not None:
                    d = np.linalg.norm(ca - cb)
                    if d < 12.0:
                        dists['a_d'].append(round(d, 2))
    
    result = {}
    for k in dists:
        if dists[k]:
            result[k] = {
                'mean': round(np.mean(dists[k]), 2),
                'std': round(np.std(dists[k]), 2),
                'min': round(min(dists[k]), 2),
                'max': round(max(dists[k]), 2),
                'values': dists[k],
                'details': details.get(k, [])
            }
    return result

print('Core packing function loaded.')

In [ ]:
# === 분석 3: Crossing Angle (DSSP helix-only PCA) ===

def calculate_crossing_angle(pdb_file):
    parser = PDBParser(QUIET=True)
    structure = parser.get_structure('d', pdb_file)
    model = structure[0]
    chains = list(model.get_chains())
    if len(chains) < 2:
        return None
    
    helix_res = {0: set(), 1: set()}
    dssp_used = False
    try:
        from Bio.PDB.DSSP import DSSP
        dssp = DSSP(model, pdb_file, dssp='mkdssp')
        for key, val in dssp:
            if val[2] == 'H':
                ci = 0 if key[0] == chains[0].id else 1
                helix_res[ci].add(key[1][1])
        dssp_used = True
    except:
        for ci, ch in enumerate(chains):
            for r in ch: helix_res[ci].add(r.get_id()[1])
    
    def get_axis(chain, h_set):
        ca = [r['CA'].get_vector().get_array() for r in chain if r.get_id()[1] in h_set and 'CA' in r]
        if len(ca) < 5: return None
        ca = np.array(ca)
        centered = ca - ca.mean(axis=0)
        eigvals, eigvecs = np.linalg.eigh(np.cov(centered.T))
        return eigvecs[:, np.argmax(eigvals)]
    
    ax_a = get_axis(chains[0], helix_res[0])
    ax_b = get_axis(chains[1], helix_res[1])
    if ax_a is None or ax_b is None:
        return None
    
    dot = np.dot(ax_a, ax_b)
    angle = np.degrees(np.arccos(min(1.0, abs(dot))))
    return {
        'crossing_angle': round(angle, 1),
        'orientation': 'parallel' if dot > 0 else 'antiparallel',
        'dssp_used': dssp_used
    }

print('Crossing angle function loaded.')

In [ ]:
# === 분석 4: Oligomeric State (C 그룹 dimer 유지 확인) ===

def check_oligomeric_state(pdb_file):
    from scipy.spatial.distance import cdist
    parser = PDBParser(QUIET=True)
    structure = parser.get_structure('d', pdb_file)
    chains = list(structure[0].get_chains())
    if len(chains) < 2:
        return {'state': 'MONOMER', 'chains': 1}
    
    ca_a = np.array([r['CA'].get_vector().get_array() for r in chains[0] if 'CA' in r])
    ca_b = np.array([r['CA'].get_vector().get_array() for r in chains[1] if 'CA' in r])
    dm = cdist(ca_a, ca_b)
    
    min_d = dm.min()
    c8 = int((dm < 8.0).sum())
    c5 = int((dm < 5.0).sum())
    cd = round(np.linalg.norm(ca_a.mean(0) - ca_b.mean(0)), 2)
    
    if min_d > 12.0: state = 'SEPARATED'
    elif c8 < 5: state = 'WEAK DIMER'
    elif c5 > 10: state = 'TIGHT DIMER'
    else: state = 'DIMER'
    
    return {'state': state, 'chains': 2, 'min_dist': round(min_d,2),
            'contacts_8A': c8, 'contacts_5A': c5, 'center_dist': cd}

print('Oligomeric state function loaded.')

In [ ]:
# === 분석 5: SASA (곤감리건별) ===

def analyze_sasa(pdb_file, sequence):
    from Bio.PDB.SASA import ShrakeRupley
    parser = PDBParser(QUIET=True)
    structure = parser.get_structure('d', pdb_file)
    sr = ShrakeRupley()
    sr.compute(structure[0], level='R')
    
    max_sasa = {'A':129,'R':274,'N':195,'D':193,'C':167,'E':223,'Q':225,
                'G':104,'H':224,'I':197,'L':201,'K':236,'M':224,'F':240,
                'P':159,'S':155,'T':172,'W':285,'Y':263,'V':174}
    
    by_type = {'gon':[],'gam':[],'ri':[],'geon':[]}
    seq2 = sequence + sequence
    for i, res in enumerate(structure[0].get_residues()):
        if i < len(seq2):
            aa = seq2[i]
            t = GGRL_MAP.get(aa)
            if t: by_type[t].append(res.sasa / max_sasa.get(aa, 200))
    
    result = {}
    for t in ['gon','gam','ri','geon']:
        if by_type[t]:
            result[t] = {'mean': round(np.mean(by_type[t]),3), 'n': len(by_type[t])}
    
    if 'gon' in result and 'geon' in result:
        ri_mean = result.get('ri',{}).get('mean', 999)
        result['naitae'] = result['gon']['mean'] < ri_mean < result['geon']['mean']
    return result

print('SASA function loaded.')

---
## 종합 분석 (5개 모델 평균, GenSpark 방식)

In [ ]:
# Step 5: 전체 모델 분석 (top-1 + 5모델 평균)

all_results = {}    # top-1 결과
avg_results = {}    # 5모델 평균

for name in SEQUENCES:
    pdbs = pdb_files.get(name, [])
    if not pdbs:
        print(f'⚠️ {name}: PDB not found')
        continue
    
    seq = SEQUENCES[name]
    print(f'\n{"="*60}')
    print(f'=== {name} ({len(pdbs)} models) ===')
    print(f'{"="*60}')
    
    model_results = []
    
    for mi, pdb in enumerate(pdbs):
        print(f'\n  --- Model {mi+1}: {os.path.basename(pdb)} ---')
        
        try:
            oligo = check_oligomeric_state(pdb)
            sb = detect_salt_bridges(pdb, seq)
            cp = analyze_core_packing(pdb, seq)
            ca = calculate_crossing_angle(pdb)
            sasa = analyze_sasa(pdb, seq)
            
            mr = {'oligo': oligo, 'sb': sb, 'cp': cp, 'ca': ca, 'sasa': sasa}
            model_results.append(mr)
            
            # 요약 출력
            print(f'    State: {oligo["state"]}')
            if sb: print(f'    Salt bridges: {sb["regular_count"]} regular, {sb["irregular_count"]} irreg')
            if cp and 'd_d' in cp: print(f'    d↔d\': mean={cp["d_d"]["mean"]}Å std={cp["d_d"]["std"]}Å')
            if ca: print(f'    Crossing: {ca["crossing_angle"]}° ({ca["orientation"]})')
        except Exception as e:
            print(f'    Error: {e}')
    
    # top-1
    if model_results:
        all_results[name] = model_results[0]
    
    # 5모델 평균
    if len(model_results) > 1:
        avg = {}
        # salt bridge 평균
        sb_reg = [m['sb']['regular_count'] for m in model_results if m['sb']]
        if sb_reg: avg['sb_regular_mean'] = round(np.mean(sb_reg), 1)
        # d↔d' 평균
        dd = [m['cp']['d_d']['mean'] for m in model_results if m['cp'] and 'd_d' in m['cp']]
        if dd: avg['dd_mean'] = round(np.mean(dd), 2)
        # crossing 평균
        ca_vals = [m['ca']['crossing_angle'] for m in model_results if m['ca']]
        if ca_vals: avg['crossing_mean'] = round(np.mean(ca_vals), 1)
        
        avg_results[name] = avg
        print(f'\n  📊 5-model average:')
        for k, v in avg.items():
            print(f'    {k}: {v}')

print('\n' + '='*60)
print('Analysis complete!')

---
## 판정

In [ ]:
# Step 6: 자동 판정

print('='*60)
print('Phase 5b-2 ColabFold Multimer — 최종 판정')
print('='*60)

r = all_results

# [1] 곤 코어
print('\n[1] 곤 코어 원리')
try:
    b_dd = r['B_GON']['cp']['d_d']['mean']
    a_dd = r['A_WT']['cp']['d_d']['mean']
    c_dd = r['C_GEON']['cp']['d_d']['mean']
    gon_pass = b_dd < a_dd < c_dd
    print(f'  d↔d\': B={b_dd}Å < A={a_dd}Å < C={c_dd}Å → {"PASS ✅" if gon_pass else "FAIL ❌"}')
except Exception as e:
    print(f'  데이터 부족: {e}')
    gon_pass = False

# [2] 건 표면
print('\n[2] 건 표면 원리')
try:
    a_sb = r['A_WT']['sb']['regular_count']
    b_sb = r['B_GON']['sb']['regular_count']
    c_sb = r['C_GEON']['sb']['regular_count']
    c_irr = r['C_GEON']['sb']['irregular_count']
    geon_pass = (a_sb >= b_sb)
    print(f'  Regular: A={a_sb}, B={b_sb}, C={c_sb} (irreg={c_irr})')
    print(f'  A≥B: {"PASS ✅" if geon_pass else "FAIL ❌"}')
    if c_irr > c_sb:
        print(f'  ⚠️ C의 비정규 salt bridge > 정규 → 레지스트리 실패 가능성')
except Exception as e:
    print(f'  데이터 부족: {e}')
    geon_pass = False

# [3] 균형
print('\n[3] 균형 설계 조건')
try:
    a_ang = r['A_WT']['ca']['crossing_angle']
    balance_pass = 15 <= a_ang <= 28
    print(f'  WT crossing: {a_ang}° → {"PASS ✅" if balance_pass else "FAIL ❌"}')
except Exception as e:
    print(f'  데이터 부족: {e}')
    balance_pass = False

# [4] C dimer
print('\n[4] C 그룹 dimer 유지')
try:
    cs = r['C_GEON']['oligo']
    print(f'  State: {cs["state"]}, contacts(8Å): {cs["contacts_8A"]}, center: {cs["center_dist"]}Å')
    if 'SEPARATED' in cs['state']:
        print('  → monomer-like collapse ❌')
    elif 'WEAK' in cs['state']:
        print('  → 약한 dimer (건 과다 → 불안정) ⚠️')
    else:
        print('  → dimer 유지 ✅')
except Exception as e:
    print(f'  데이터 부족: {e}')

# [5] SASA 나이테
print('\n[5] SASA 나이테')
try:
    sa = r['A_WT']['sasa']
    print(f'  곤={sa["gon"]["mean"]}, 리={sa.get("ri",{}).get("mean","N/A")}, 건={sa["geon"]["mean"]}')
    print(f'  곤<리<건: {"PASS ✅" if sa.get("naitae") else "FAIL ❌"}')
except Exception as e:
    print(f'  데이터 부족: {e}')

# 종합
print('\n' + '='*60)
tp = sum([gon_pass, geon_pass, balance_pass])
print(f'핵심 판정: {tp}/3 PASS')
if tp == 3:
    print('🎉 Phase 5b-2 완료! → 곤/건/균형 3원리 고정')
else:
    print(f'⚠️ {3-tp}개 항목 추가 검증 필요')
print('='*60)

---
## 시각화

In [ ]:
# Step 7: matplotlib 3패널
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 3, figsize=(15, 5))
groups = ['A_WT', 'B_GON', 'C_GEON']
labels = ['A (WT)', 'B (곤↑)', 'C (건↑)']
colors = ['#3498db', '#e74c3c', '#2ecc71']

# Plot 1: d↔d'
ax = axes[0]
vals, errs = [], []
for g in groups:
    if g in all_results and all_results[g]['cp'] and 'd_d' in all_results[g]['cp']:
        vals.append(all_results[g]['cp']['d_d']['mean'])
        errs.append(all_results[g]['cp']['d_d']['std'])
    else: vals.append(0); errs.append(0)
ax.bar(labels, vals, yerr=errs, color=colors, capsize=5, edgecolor='black', lw=0.5)
ax.axhspan(5, 8, alpha=0.1, color='green', label='Normal 5-8Å')
ax.set_ylabel('d↔d\' Cβ-Cβ (Å)'); ax.set_title('Core Packing'); ax.legend()

# Plot 2: Salt bridges
ax = axes[1]
sr, si = [], []
for g in groups:
    if g in all_results and all_results[g]['sb']:
        sr.append(all_results[g]['sb']['regular_count'])
        si.append(all_results[g]['sb']['irregular_count'])
    else: sr.append(0); si.append(0)
x = np.arange(3)
ax.bar(x, sr, 0.4, label='Regular (e↔g\')', color=colors, edgecolor='black', lw=0.5)
ax.bar(x, si, 0.4, bottom=sr, label='Irregular', alpha=0.4, color='gray', edgecolor='black', lw=0.5)
ax.set_xticks(x); ax.set_xticklabels(labels)
ax.set_ylabel('Count'); ax.set_title('Salt Bridges'); ax.legend()

# Plot 3: Crossing angle
ax = axes[2]
angs = [all_results[g]['ca']['crossing_angle'] if g in all_results and all_results[g]['ca'] else 0 for g in groups]
ax.bar(labels, angs, color=colors, edgecolor='black', lw=0.5)
ax.axhspan(18, 25, alpha=0.15, color='green', label='Normal 18-25°')
ax.set_ylabel('Angle (°)'); ax.set_title('Crossing Angle'); ax.legend()

plt.tight_layout()
plt.savefig('gcn4_colabfold/phase5b2_summary.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Step 8: 3D 구조 시각화 (GenSpark 방식)
try:
    import py3Dmol
    
    for name in SEQUENCES:
        pdbs = pdb_files.get(name, [])
        if not pdbs:
            continue
        
        pdb_path = pdbs[0]  # top-ranked model
        print(f'\n=== {name}: {os.path.basename(pdb_path)} ===')
        
        with open(pdb_path) as f:
            pdb_str = f.read()
        
        view = py3Dmol.view(width=800, height=500)
        view.addModel(pdb_str, 'pdb')
        
        # Chain A = blue, Chain B = red
        view.setStyle({'chain': 'A'}, {'cartoon': {'color': '#3498db', 'opacity': 0.85}})
        view.setStyle({'chain': 'B'}, {'cartoon': {'color': '#e74c3c', 'opacity': 0.85}})
        
        # a,d 코어 잔기 (stick)
        registry = get_heptad_registry(len(SEQUENCES[name]))
        positions = get_positions_by_role(registry)
        core_resi = [str(p+1) for p in positions['a'] + positions['d']]
        if core_resi:
            resi_str = ','.join(core_resi)
            view.addStyle({'resi': core_resi, 'chain': 'A'},
                         {'stick': {'color': '#f39c12', 'radius': 0.15}})
            view.addStyle({'resi': core_resi, 'chain': 'B'},
                         {'stick': {'color': '#f39c12', 'radius': 0.15}})
        
        # e,g salt bridge 잔기 (line)
        salt_resi = [str(p+1) for p in positions['e'] + positions['g']]
        if salt_resi:
            view.addStyle({'resi': salt_resi, 'chain': 'A'},
                         {'stick': {'color': '#9b59b6', 'radius': 0.1}})
            view.addStyle({'resi': salt_resi, 'chain': 'B'},
                         {'stick': {'color': '#9b59b6', 'radius': 0.1}})
        
        view.zoomTo()
        view.show()

except ImportError:
    print('py3Dmol 미설치. pip install py3Dmol')
    print('PyMOL, ChimeraX 등으로 PDB 파일을 열어 확인하세요.')

In [ ]:
# Step 9: 결과 저장

output = {
    'sequences': SEQUENCES,
    'top1_results': {},
    'average_results': avg_results
}

for name in all_results:
    r = all_results[name]
    output['top1_results'][name] = {
        'oligomeric': r['oligo'],
        'salt_bridges': r['sb'],
        'core_packing': r['cp'],
        'crossing_angle': r['ca'],
        'sasa': r['sasa']
    }

with open('gcn4_colabfold/phase5b2_final_results.json', 'w') as f:
    json.dump(output, f, indent=2, default=str)

print('Results saved to gcn4_colabfold/phase5b2_final_results.json')
print('\n--- 파일 목록 ---')
for f in glob.glob('gcn4_colabfold/*'):
    print(f'  {f}')